In [1]:
import findspark
findspark.init(r'C:\Program Files\spark-3.5.2-bin-hadoop3\spark-3.5.2-bin-hadoop3')
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("My Session Titanic").getOrCreate()

In [4]:
data = spark.read.csv(r'C:\Users\Arnab\Python-and-Spark-for-Big-Data-master\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Logistic_Regression\titanic.csv'
                     ,inferSchema=True, header=True)

In [5]:
data.head()

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S')

In [6]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [7]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [35]:
my_cols = data.select(['Pclass','Sex','Age','Parch','Fare','Embarked','Survived'])

In [36]:
my_cols.show()

+------+------+----+-----+-------+--------+--------+
|Pclass|   Sex| Age|Parch|   Fare|Embarked|Survived|
+------+------+----+-----+-------+--------+--------+
|     3|  male|22.0|    0|   7.25|       S|       0|
|     1|female|38.0|    0|71.2833|       C|       1|
|     3|female|26.0|    0|  7.925|       S|       1|
|     1|female|35.0|    0|   53.1|       S|       1|
|     3|  male|35.0|    0|   8.05|       S|       0|
|     3|  male|NULL|    0| 8.4583|       Q|       0|
|     1|  male|54.0|    0|51.8625|       S|       0|
|     3|  male| 2.0|    1| 21.075|       S|       0|
|     3|female|27.0|    2|11.1333|       S|       1|
|     2|female|14.0|    0|30.0708|       C|       1|
|     3|female| 4.0|    1|   16.7|       S|       1|
|     1|female|58.0|    0|  26.55|       S|       1|
|     3|  male|20.0|    0|   8.05|       S|       0|
|     3|  male|39.0|    5| 31.275|       S|       0|
|     3|female|14.0|    0| 7.8542|       S|       0|
|     2|female|55.0|    0|   16.0|       S|   

In [37]:
my_final_data = my_cols.dropna()

In [38]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [39]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVector')


In [40]:
embard_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIndex')
embard_encoder = OneHotEncoder(inputCol='EmbarkedIndex', outputCol='EmbarkedVector')

In [41]:
from pyspark.ml.classification import LogisticRegression

In [42]:
from pyspark.ml import Pipeline

In [43]:
assembler = VectorAssembler(inputCols=['Pclass','SexVector','Age','Parch','Fare','EmbarkedVector'], outputCol='features')

In [44]:
my_log_model = LogisticRegression(featuresCol='features', labelCol='Survived')

In [45]:
pipeline = Pipeline(stages=[gender_indexer, embard_indexer,gender_encoder,embard_encoder,assembler,my_log_model])

In [70]:
train_data, test_data = my_final_data.randomSplit([0.69,0.31])

In [71]:
fit_model = pipeline.fit(train_data)

In [72]:
result = fit_model.transform(test_data)

In [73]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [74]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [75]:
result.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
+--------+----------+
only showing top 20 rows



In [76]:
AUC = my_eval.evaluate(result)

In [77]:
print(AUC)

0.7784272465123528
